In [29]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import types
from pprint import pprint

@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__(self):
        return f"{self.name} ({self.date})"

In [34]:
class HolidayList:
    "HolidayList class"
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            self.innerHolidays.append(holidayObj)
            print(f"Added holiday {holidayObj}")

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            if holiday.name == HolidayName and holiday.date == Date:
                return holiday
            else:
                print(f"{holiday} is not in list.")

    def removeHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            if holiday.name == HolidayName and holiday.date == Date:
                self.innerHolidays.remove(holiday)
                print(f"{holiday} was removed from the list.")
            else:
                print(f"{holiday} is not in list.")

    def read_json(self, filelocation):
        with open(filelocation, "r") as jsonfile:
            data = json.load(jsonfile)
            for i in range(len(data["holidays"])):
                holiday = Holiday(data["holidays"][i]["name"], data["holidays"][i]["date"])
                HolidayList.addHoliday(self, holiday)

    def save_to_json(self, filelocation):
        json_list = []
        json_dict = {"holidays": json_list}
        with open(filelocation, "w") as jsonfile:
            for holiday in self.innerHolidays:
                json_list.append(holiday.__dict__)
            json.dump(json_dict, jsonfile, indent=3)

    def scrapeHolidays(self):
        def getHTML(url):
            response = requests.get(url)
            return response.text
        # Scrape holidays from 2020-2024
        for year in range(2020,2025):
            year = str(year)
            html = getHTML("https://www.timeanddate.com/calendar/custom.html?year=" + year +"&country=1&cols=3&df=1&hol=33554809")
            soup = BeautifulSoup(html,'html.parser')
            holiday_data = soup.find('table', attrs = {'class':'cht lpad'}).find('tbody')
            for row in holiday_data:
                items = row.find_all('tr')
                holiday_dict = {}
                holiday_dict['name'] = row.find('td').find_next('td').string
                holiday_dict['date'] = f"{row.find('td').string}, {year}"
                # Check to see if name and date of holiday is in innerHolidays array. If it is, don't add it to list
                if holiday_dict['name'] in self.innerHolidays and holiday_dict['date'] in self.innerHolidays:
                    continue
                else:
                    self.innerHolidays.append(holiday_dict)

    


In [35]:
holiday1 = Holiday("New Year's Day", "2022-01-01")
#print(holiday1)

holidaylist = HolidayList()

holidaylist.addHoliday(holiday1)
#holidaylist.findHoliday("New Year's Day", "2022-01-01")
#holidaylist.removeHoliday("New Year's Day", "2022-01-01")

#holidaylist.read_json("holidays.json")
#holidaylist.save_to_json("holidays.json")

#print(type(holidaylist.innerHolidays[0]))
holidaylist.scrapeHolidays()



Added holiday New Year's Day (2022-01-01)
[Holiday(name="New Year's Day", date='2022-01-01'), {'name': "New Year's Day", 'date': 'Jan 1, 2020'}, {'name': 'Stephen Foster Memorial Day', 'date': 'Jan 13, 2020'}, {'name': 'Martin Luther King Jr. Day', 'date': 'Jan 20, 2020'}, {'name': 'Chinese New Year', 'date': 'Jan 25, 2020'}, {'name': 'Kansas Day (Kansas)', 'date': 'Jan 29, 2020'}, {'name': 'National Freedom Day', 'date': 'Feb 1, 2020'}, {'name': 'First Day of Black History Month', 'date': 'Feb 1, 2020'}, {'name': 'Groundhog Day', 'date': 'Feb 2, 2020'}, {'name': 'Super Bowl', 'date': 'Feb 2, 2020'}, {'name': 'National Wear Red Day', 'date': 'Feb 7, 2020'}, {'name': "Valentine's Day", 'date': 'Feb 14, 2020'}, {'name': "Presidents' Day", 'date': 'Feb 17, 2020'}, {'name': 'Shrove Tuesday/Mardi Gras', 'date': 'Feb 25, 2020'}, {'name': "St. David's Day", 'date': 'Mar 1, 2020'}, {'name': "First Day of Women's History Month", 'date': 'Mar 1, 2020'}, {'name': 'First Day of Irish American Heri

In [ ]:
import requests

url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}

headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"
    }

weather = requests.request("GET", url, headers=headers, params=querystring)
weather_dict = weather.json()

print(weather_dict)

print(weather_dict["list"][0]["weather"][0]["main"])

{'city': {'id': 5128581, 'name': 'New York', 'coord': {'lon': -74.006, 'lat': 40.7143}, 'country': 'US', 'population': 8175133, 'timezone': -18000}, 'cod': '200', 'message': 20.1974126, 'cnt': 7, 'list': [{'dt': 1641747600, 'sunrise': 1641730779, 'sunset': 1641764775, 'temp': {'day': 36.16, 'min': 25.95, 'max': 38.46, 'night': 38.46, 'eve': 35.73, 'morn': 31.69}, 'feels_like': {'day': 26.4, 'night': 30.43, 'eve': 28, 'morn': 22.37}, 'pressure': 1022, 'humidity': 77, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'speed': 16.6, 'deg': 212, 'gust': 37.58, 'clouds': 100, 'pop': 0.93, 'rain': 1.89}, {'dt': 1641834000, 'sunrise': 1641817166, 'sunset': 1641851236, 'temp': {'day': 28.81, 'min': 26.04, 'max': 37.96, 'night': 26.04, 'eve': 28.89, 'morn': 27.79}, 'feels_like': {'day': 17.47, 'night': 14.29, 'eve': 19.45, 'morn': 17.8}, 'pressure': 1022, 'humidity': 43, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d